# **<p style="text-align: center;">Aprendizaje por refuerzo - Ingeniería en Inteligencia Artificial</p>**
## **<p style="text-align: center;">Trabajo práctico Final - SAC</p>**

#### <p style="text-align: center;">Pettinari Fausto, Schuemer Ignacio, Torres Santiago </p>
#### <p style="text-align: center;">Profesores: Claudio Pose, Gabriel Torre, Nicolás Romero, Tomás Chimenti</p>

# **Mujoco Inverted Pendulum Enviroment**

In [1]:
import sys
sys.path.append('..')

In [2]:
import gymnasium as gym

In [3]:
env = gym.make('InvertedPendulum-v5', reset_noise_scale=0.1, max_episode_steps=250)

#### **Logging & Debugging (Random Agent)**

In [4]:
from torch.utils.tensorboard import SummaryWriter
from sac.random_agent import random_agent_loop

In [ ]:
seed = 42
num_episodes = 400

## **SAC Agent**

In [6]:
import yaml
from sac.agent import SAC

Bad value in file PosixPath('/home/heisenberg/Documentos/GitHub/RL-SAC/sac/utils/custom.mplstyle'), line 4 ("axes.prop_cycle : cycler('color', ['#0000FF', '#FFA500', '#006400', '#8B0000']) + cycler('linestyle', ['-'])"): Key axes.prop_cycle: "cycler('color', ['" is not a valid cycler construction: unterminated string literal (detected at line 1) (<unknown>, line 1)


In [7]:
with open('configs/inverted_pendulum.yaml', 'r') as f:
    config = yaml.safe_load(f)

### **TRAIN**

In [30]:
SAC_agent = SAC(env=env, config=config)
SAC_agent.run_training_loop(num_episodes=num_episodes, tqdm_disable=False, print_rewards=False)

  0%|          | 0/400 [00:00<?, ?it/s]

100%|██████████| 400/400 [11:37<00:00,  1.74s/it]

Agent saved to runs/InvertedPendulum-v5/SAC/sac-inverted-pendulum-2025_11_29-14_54_49/sac_agent.pth


{'total_episodes': 400,
 'best_avg_return': np.float64(250.0),
 'final_avg_return': np.float64(250.0)}

#### **Eval trained Agent**

In [ ]:
SAC_agent.eval_agent(num_episodes=1, render_mode="human")

### **Load trained agent & Eval**

In [8]:
# Load trained agent and run in environment
config['logger']['enabled'] = False  # Disable logging for loaded agent
SAC_agent_loaded = SAC(env=env, config=config)
SAC_agent_loaded.load_agent('runs/InvertedPendulum-v5/SAC/sac-inverted-pendulum-2025_11_25-00_05_14/sac_agent.pth')

In [19]:
SAC_agent_loaded.eval_agent(num_episodes=1, render_mode="human")

Creating new environment for evaluation with render_mode='human'


  0%|          | 0/1 [00:00<?, ?it/s]/home/ignacio/miniconda3/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)
100%|██████████| 1/1 [00:04<00:00,  4.07s/it]


1000.0

## **SB3 SAC Agent**


In [34]:
from stable_baselines3 import SAC as SB3_SAC
from sac.utils.stable_baseline_params import get_sb3_sac_params
from sac.utils.stable_baseline_logger import RobustEpisodeLogger

sb3_params = get_sb3_sac_params(env, config, seed, env_id="InvertedPendulum-v5")

writer = SummaryWriter(f"runs/InvertedPendulum-v5/sac_sb3-sin_param/")
sb3_sac = SB3_SAC(policy="MlpPolicy", env=env, seed=seed)
callback = RobustEpisodeLogger(writer, max_episodes=num_episodes, save_dir="runs/InvertedPendulum-v5/sac_sb3-sin_param/")

sb3_sac.learn(
    total_timesteps=num_episodes * env.spec.max_episode_steps,
    callback=callback
)

writer.close()
env.close()

Reached max episodes → early stopping training.
